In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.formula.api import ols

In [18]:
%matplotlib inline
import matplotlib
from matplotlib import style
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## 반복문 RFR 분석

In [13]:
plt.rcParams['font.family'] = 'NanumGothic'

In [14]:
df = pd.read_csv('data/unmatch_data1.csv',encoding='euc-kr').drop(columns=['Unnamed: 0'])
df.head()

,location,year,month,lon,lat,height,avg_tem,precip,wind,min_tem,hPa,sun,g_tem,start,climax
0,속초,1989,1,128.5647,38.2509,17.53,1.5,165.9,2.8,-1.5,1023.7,111.6,0.8,286,297
1,속초,1989,2,128.5647,38.2509,17.53,2.7,65.3,2.6,-0.5,1022.4,154.2,1.9,286,297
2,속초,1989,3,128.5647,38.2509,17.53,5.9,121.7,3.2,2.6,1017.0,175.4,6.1,286,297
3,속초,1989,4,128.5647,38.2509,17.53,12.4,6.2,3.2,7.5,1013.3,228.1,14.2,286,297
4,속초,1989,5,128.5647,38.2509,17.53,16.5,66.8,3.5,12.5,1010.6,253.7,18.7,286,297


In [15]:
lo = ['속초', '북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉',
       '동해', '서울', '인천', '원주', '울릉도', '수원', '영월', '충주', '서산', '울진', '청주',
       '대전', '추풍령', '안동', '상주', '포항', '군산', '대구', '전주', '울산', '창원', '광주',
       '부산', '통영', '목포', '여수', '흑산도', '완도', '고창', '순천', '홍성', '제주', '고산',
       '성산', '서귀포', '진주', '강화', '양평', '이천', '인제', '홍천', '태백', '제천', '보은',
       '천안', '보령', '부여', '금산', '부안', '임실', '정읍', '남원', '장수', '고창군', '장흥',
       '해남', '고흥', '봉화', '영주', '문경', '영덕', '의성', '구미', '영천', '거창', '합천',
       '밀양', '산청', '거제', '남해']


In [16]:
cols = ['precip', 'wind', 'min_tem', 'hPa', 'sun', 'g_tem', 'start']

In [19]:
for c in range(len(cols)):
    locat = []
    er_mean = []
    er_max = []

    for i in range(len(lo)):
        cond = (df.month == 10) & (df.location == f'{lo[i]}')

        test = df[cond]

        X = test.iloc[:,[c+7]]
        y = test.climax

        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.4, random_state=1)

        forest = RandomForestRegressor(n_estimators=500,criterion='squared_error',n_jobs=-1)

        forest.fit(X_train,y_train)

        error = []
        location = []
        pred_data = []

        for j in range(len(lo)):    
            cond1 = (df.location == f'{lo[j]}')

            pred = forest.predict(pd.DataFrame(df[cond1].iloc[:,c+7]))
            location.append(lo[j])
            error.append(np.sqrt(mean_squared_error(pd.DataFrame(df[cond1].iloc[:,14]),pred)))
            pred_data.append(pred)
        dc = pd.DataFrame({'error':error,'pred_data':pred_data})
        dc.to_csv(f'local_model/{df.columns[c+7]}_{lo[i]}_RFL.csv',encoding='cp949',index=False)
        print(f'지역: {lo[i]} 기준 | data: {df.columns[c+7]} 파일 저장')

        locat.append(lo[i])
        er_mean.append(dc.error.mean())
        er_max.append(dc.error.max())
    pd.DataFrame({'location':lo,'er_mean':er_mean,'er_max':er_max}).to_csv(f'RFL/{cols[c]}_RFL.csv',encoding='cp949',index=False)
    print(f'{df.columns[c+7]} 컬럼 예측완료')

C:\Java\miniconda3\envs\.py39\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


ValueError: Found input variables with inconsistent numbers of samples: [312, 26]

In [34]:
lo = ['속초', '북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉']

In [42]:
cond = (df.month == 10) & (df.location == '북춘천')

test = df[cond]

X = test.iloc[:,[c+7]]
y = test.climax

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.4, random_state=1)

forest = RandomForestRegressor(n_estimators=500,criterion='squared_error',n_jobs=-1)

forest.fit(X_train,y_train)

error = []
location = []
pred_data = []

for j in range(len(lo)):    
    cond1 = (df.location == f'{lo[j]}') & (df.month == 10)

    pred = forest.predict(pd.DataFrame(df[cond1].iloc[:,7]))
    location.append(lo[j])
    error.append(np.sqrt(mean_squared_error(pd.DataFrame(df[cond1].iloc[:,14]),pred)))
    ar = pd.DataFrame({'climax':df[cond1].climax,'pred':pred})
    ar.to_csv(f'local_model/{lo[j]}_pred.csv',encoding='cp949',index=False)
dc = pd.DataFrame({'location':location,'error':error})
dc    

,location,error
0,속초,6.004781
1,북춘천,2.762860
2,철원,9.997155
3,동두천,4.831989
4,파주,6.838921
5,대관령,16.576315
6,춘천,5.722811
7,백령도,8.953039
8,북강릉,6.156256
9,강릉,10.978706


### 튜닝1

In [26]:
# cond = (df.month == 10) & (df.location == '구미')

# test = df[cond]

# X = test.iloc[:,[7]]
# y = test.climax

# X_train, X_test, y_train, y_test = \
#     train_test_split(X, y, test_size=0.4, random_state=1)

# forest = RandomForestRegressor()

# forest.fit(X_train,y_train)

# error = []
# location = []

# for j in range(len(lo)):    
#     cond1 = (df.location == f'{lo[j]}')

#     pred = forest.predict(pd.DataFrame(df[cond1].iloc[:,c+7]))
#     location.append(lo[j])
#     error.append(np.sqrt(mean_squared_error(pd.DataFrame(df[cond1].iloc[:,14]),pred)))
# dc = pd.DataFrame({'location':location,'error':error})

In [27]:
dc

,location,error
0,속초,7.165810
1,북춘천,6.072853
2,철원,9.733922
3,동두천,7.748634
4,파주,8.396967
...,...,...
75,합천,8.749334
76,밀양,10.040271
77,산청,9.059125
78,거제,21.145168


## 결과 확인

In [5]:
avg_tem = pd.read_csv('RFL/avg_tem_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
g_tem = pd.read_csv('RFL/g_tem_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
hPa = pd.read_csv('RFL/hPa_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
min_tem = pd.read_csv('RFL/min_tem_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
precip = pd.read_csv('RFL/precip_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
start = pd.read_csv('RFL/start_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
sun = pd.read_csv('RFL/sun_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])
wind = pd.read_csv('RFL/wind_RFL.csv',encoding='cp949').drop(columns=['Unnamed: 0'])

In [8]:
print('avg_tem',avg_tem.er_mean.mean(),avg_tem.er_max.mean())
print('g_tem',g_tem.er_mean.mean(),g_tem.er_max.mean())
print('hPa',hPa.er_mean.mean(),hPa.er_max.mean())
print('min_tem',min_tem.er_mean.mean(),min_tem.er_max.mean())
print('precip',precip.er_mean.mean(),precip.er_max.mean())
print('start',start.er_mean.mean(),start.er_max.mean())
print('sun',sun.er_mean.mean(),sun.er_max.mean())
print('wind',wind.er_mean.mean(),wind.er_max.mean())

avg_tem 11.386747747012993 23.715085459285596
g_tem 4.828135526826348 4.82813552682635
hPa 4.9702582410185 4.970258241018503
min_tem 4.546154788155112 4.546154788155116
precip 4.755841592953094 4.755841592953095
start 3.54159021121864 3.5415902112186446
sun 4.749188918179028 4.749188918179028
wind 5.056216622341594 5.056216622341602


In [9]:
avg_tem.er_mean

0     11.532877
1      9.123461
2     10.109389
3      9.939734
4      9.708658
        ...    
75    10.262107
76     9.327361
77     9.215241
78    13.655315
79    18.469084
Name: er_mean, Length: 80, dtype: float64